# Data Science Final Project - EE 379k - sp18 - 

# Shahshank Kambhampati - skk834, Shrikara Murthy - svm456, Pranav Harathi - , Neil Charles - 

# Job Satisfaction Prediction

## Imports

### All hail lord and savior XGB

In [26]:
import pandas as pd
import re
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from scipy import stats
from sklearn.linear_model import RidgeCV, LassoCV, LogisticRegression, Ridge, Lasso
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomTreesEmbedding, AdaBoostClassifier, AdaBoostRegressor, RandomForestRegressor, RandomForestClassifier, VotingClassifier
from sklearn.metrics import make_scorer, roc_auc_score, accuracy_score, mean_squared_error
from sklearn.model_selection import LeaveOneOut, train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_selection import RFE,RFECV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.feature_selection import SelectKBest,  chi2
from sklearn.svm import SVR

from __future__ import print_function
%config inlinebackend.figure_format = 'retina' 
%matplotlib inline

# Initial Observations about the Data and Pre-Processing

In [27]:
number = re.compile('[\d,]+')
def get_first_number(val):
    matched = number.match(str(val))
    if matched:
        return int(matched.group())
    else:
        return np.nan

satisfaction_strs = {
    'I love my job': 10,
    'I\'m somewhat satisfied with my job': 7.5,
    'I\'m neither satisfied nor dissatisfied with my job': 5,
    'I\'m somewhat dissatisfied with my job': 2.5,
    'I hate my job': 0,
}

binary_labels = [
    'Lang & Tech',
    'Training & Education',
    'How can companies improve interview process',
    'Why try Stack Overflow Careers',
    'Most important aspect of new job opportunity',
    'Most annoying about job search',
    'Appealing message traits',
    'Most urgent info about job opportunity',
    'Who do you want to communicate with about a new job opportunity',
    'Why use Stack Overflow',
    'Why answer',
    'Source control used',
]

numeric_labels = [
    'Age',
    'Years IT / Programming Experience',
    'Compensation: midpoint'
]

yes_no_labels = [
    'Changed Jobs in last 12 Months'
]


In [28]:
# 2015 preproc
data = pd.read_csv('data/2015.csv')
to_drop = [label for label in data if 'write-in' in label.lower()]

data['Job Satisfaction'] = data['Job Satisfaction']\
                                    .map(satisfaction_strs).astype('float')
data = data[data['Job Satisfaction'].notnull()]
to_drop.append('Country')
to_drop.append('Compensation')

data['Age'] = data['Age'].map(get_first_number).astype('float')
data['gender_M'] = (data['Gender'] == 'Male').astype('int8')
data['gender_F'] = (data['Gender'] == 'Female').astype('int8')
to_drop.append('Gender')
to_drop.append('Prefered Source Control')

bin_labels = [key for key in data if any(label in key for label in binary_labels)]
data[bin_labels] = data[bin_labels].apply(lambda col: col.notnull().astype('int8'))

data[numeric_labels] = data[numeric_labels].applymap(get_first_number)
data[yes_no_labels] = data[yes_no_labels]\
                                .apply(lambda col: col.map({'Yes': 1, 'No': 0}))\
                                .fillna(0)

data.drop(to_drop, axis=1, inplace=True)
data = pd.get_dummies(data)

print("Columns with NaNs:")

for key in data:
    if data[key].isnull().any():
        print (key)
data.fillna(data.mean(), inplace=True)
print ('Filling with mean of column')

Columns with NaNs:
Age
Years IT / Programming Experience
Compensation: midpoint
Filling with mean of column


In [29]:
# data contains a DataFrame with no NaNs, all numbers.
# We're trying to predict the "Job Satisfaction" column.

In [30]:
X = data.drop(['Job Satisfaction'], axis=1)
y = data['Job Satisfaction']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)#stratify=y
scoring = {'mean': make_scorer(mean_squared_error)}

In [31]:
print (data.head())

    Age  Years IT / Programming Experience  Current Lang & Tech: Android  \
0  25.0                                2.0                             0   
1  20.0                                1.0                             0   
2  20.0                                1.0                             0   
3  25.0                                6.0                             0   
4  30.0                                2.0                             0   

   Current Lang & Tech: Arduino  Current Lang & Tech: AngularJS  \
0                             0                               0   
1                             0                               0   
2                             0                               0   
3                             0                               0   
4                             0                               0   

   Current Lang & Tech: C  Current Lang & Tech: C++  \
0                       0                         0   
1                       0     

In [32]:
print (data.describe())

                Age  Years IT / Programming Experience  \
count  16004.000000                       16004.000000   
mean      28.478348                           5.839325   
std        6.955481                           3.850501   
min       20.000000                           1.000000   
25%       25.000000                           2.000000   
50%       25.000000                           6.000000   
75%       30.000000                          11.000000   
max       51.000000                          11.000000   

       Current Lang & Tech: Android  Current Lang & Tech: Arduino  \
count                  16004.000000                  16004.000000   
mean                       0.173269                      0.061110   
std                        0.378492                      0.239539   
min                        0.000000                      0.000000   
25%                        0.000000                      0.000000   
50%                        0.000000                      0.0000

In [33]:
for i in data.corr():
    print (i)

Age
Years IT / Programming Experience
Current Lang & Tech: Android
Current Lang & Tech: Arduino
Current Lang & Tech: AngularJS
Current Lang & Tech: C
Current Lang & Tech: C++
Current Lang & Tech: C++11
Current Lang & Tech: C#
Current Lang & Tech: Cassandra
Current Lang & Tech: CoffeeScript
Current Lang & Tech: Cordova
Current Lang & Tech: Clojure
Current Lang & Tech: Cloud
Current Lang & Tech: Dart
Current Lang & Tech: F#
Current Lang & Tech: Go
Current Lang & Tech: Hadoop
Current Lang & Tech: Haskell
Current Lang & Tech: iOS
Current Lang & Tech: Java
Current Lang & Tech: JavaScript
Current Lang & Tech: LAMP
Current Lang & Tech: Matlab
Current Lang & Tech: MongoDB
Current Lang & Tech: Node.js
Current Lang & Tech: Objective-C
Current Lang & Tech: Perl
Current Lang & Tech: PHP
Current Lang & Tech: Python
Current Lang & Tech: R
Current Lang & Tech: Redis
Current Lang & Tech: Ruby
Current Lang & Tech: Rust
Current Lang & Tech: Salesforce
Current Lang & Tech: Scala
Current Lang & Tech: Shar

In [34]:
model = xgb.XGBRegressor()
rfe = RFE(model, 50,2)
rfe = rfe.fit(X_train,y_train)
X_train = rfe.transform(X_train)
X_test = rfe.transform(X_test)
#rfe_preds = rfe.predict(X_test)
#print mean_squared_error(y_test,rfe_preds)


KeyboardInterrupt: 

In [36]:
parameters = {
    'n_estimators':[160]
}
modelETC = ExtraTreesRegressor()
gridETC = GridSearchCV(modelETC, param_grid=parameters,scoring = scoring, cv=5, refit = 'mean',verbose=1)
gridETC.fit(X_train,y_train)
print(gridETC.best_params_)
#print(modelETC.feature_importances_)
etc_preds = gridETC.predict(X_test)
print (mean_squared_error(y_test,etc_preds))

Fitting 5 folds for each of 1 candidates, totalling 5 fits


KeyboardInterrupt: 

In [ ]:
print mean_squared_error(y_test,etc_preds)

In [37]:
parameters = { #when use hyperthread, xgboost may become slower
              #'objective':['binary:logistic'],
             # 'booster':['dart'],
               'booster':['gbtree'],
              'learning_rate': [.1], #so called `eta` value
              'max_depth': [9],#
              'min_child_weight': [5],#
              'reg_alpha':[.1],
              'silent': [0],
              'subsample': [.7],
              'gamma':[.3],
              'colsample_bytree': [.8],
              'n_estimators': [1000], #number of trees, change it to 1000 for better results
              'seed': [1337]
}

In [46]:
xgb_model = xgb.XGBRegressor()
#gridXGB = GridSearchCV(xgb_model, param_grid=parameters,scoring = scoring, cv=5, verbose=2, refit = 'mean')
#gridXGB.fit(X_train, y_train,eval_metric='rmse')
#print(gridXGB.best_params_)
#xgb_preds = gridXGB.predict(X_test)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)
print (mean_squared_error(y_test,xgb_preds)**.5)
xgb_copy = list(xgb_preds)

2.247686633130304


In [47]:
print (sum(y_test)/len(y_test))
print (sum(xgb_preds)/len(xgb_preds))

7.5
7.492633136249195


In [48]:
def clipS(preds):
    for pred, element in enumerate(preds):
        if preds[pred] < 1.25:
            preds[pred] = 0
        elif preds[pred] < 3.75:
            preds[pred] = 2.5
        elif  preds[pred] < 6.25:
            preds[pred] = 5
        elif  preds[pred] < 8.75:
            preds[pred] = 7.5
        else: 
            preds[pred] = 10
    return preds

In [49]:
for pred, clipped, actual in zip(xgb_preds, clipS(xgb_copy), y_test ):
    print (pred, clipped, actual)

8.286079 7.5 10.0
7.927208 7.5 7.5
6.8665676 7.5 7.5
7.365894 7.5 7.5
8.49939 7.5 5.0
7.1254463 7.5 5.0
6.702565 7.5 7.5
5.842753 5 7.5
8.738859 7.5 10.0
7.841649 7.5 10.0
6.74351 7.5 7.5
7.807215 7.5 7.5
9.261417 10 7.5
7.109841 7.5 5.0
6.8010564 7.5 7.5
7.597346 7.5 10.0
6.9410725 7.5 2.5
6.5449824 7.5 0.0
7.402582 7.5 5.0
6.431654 7.5 0.0
7.1956844 7.5 7.5
7.6999984 7.5 10.0
6.1977854 5 2.5
7.982983 7.5 10.0
7.190724 7.5 7.5
6.7932687 7.5 10.0
8.369915 7.5 7.5
8.421787 7.5 7.5
7.026611 7.5 7.5
6.857605 7.5 7.5
8.973689 10 10.0
6.7475266 7.5 2.5
8.897934 10 7.5
7.8295035 7.5 10.0
9.146121 10 10.0
8.495934 7.5 7.5
6.728901 7.5 7.5
7.36117 7.5 10.0
8.701159 7.5 10.0
7.6483 7.5 10.0
7.262645 7.5 7.5
7.3512554 7.5 7.5
7.0285273 7.5 10.0
8.729461 7.5 7.5
7.8325706 7.5 7.5
8.742565 7.5 10.0
6.331672 7.5 0.0
8.884819 10 7.5
7.004472 7.5 7.5
7.0543513 7.5 5.0
7.403653 7.5 10.0
7.0553303 7.5 7.5
7.1007104 7.5 7.5
4.405138 5 5.0
6.3311124 7.5 5.0
7.363051 7.5 5.0
6.0551515 5 2.5
9.117585 10 10

7.2498374 7.5 10.0
7.0629873 7.5 10.0
6.8996253 7.5 7.5
6.8121514 7.5 10.0
7.2658896 7.5 2.5
8.852533 10 10.0
7.584811 7.5 5.0
8.762749 10 10.0
6.7032113 7.5 7.5
7.040448 7.5 7.5
8.731287 7.5 10.0
8.339863 7.5 10.0
9.299665 10 10.0
7.84971 7.5 7.5
5.7796865 5 7.5
5.215879 5 2.5
6.0454664 5 5.0
7.069505 7.5 7.5
7.026026 7.5 10.0
8.7816305 10 7.5
7.6395483 7.5 10.0
8.811912 10 7.5
7.073048 7.5 7.5
6.7585797 7.5 7.5
5.5238676 5 7.5
7.074044 7.5 7.5
8.605527 7.5 10.0
4.81072 5 2.5
6.474626 7.5 10.0
7.489467 7.5 10.0
6.297437 7.5 7.5
9.015276 10 10.0
7.1141887 7.5 2.5
8.855669 10 7.5
7.5748086 7.5 7.5
7.4217353 7.5 10.0
7.6471562 7.5 7.5
6.816087 7.5 5.0
8.472724 7.5 10.0
6.552516 7.5 7.5
7.1472735 7.5 2.5
5.7331457 5 7.5
6.8766747 7.5 2.5
5.6829224 5 2.5
9.096558 10 10.0
7.444388 7.5 7.5
8.846415 10 7.5
7.449927 7.5 7.5
7.189878 7.5 7.5
6.0751224 5 10.0
7.364566 7.5 10.0
5.6063213 5 10.0
7.286406 7.5 7.5
7.6666327 7.5 7.5
6.3354225 7.5 10.0
5.25561 5 10.0
9.825521 10 10.0
6.786795 7.5 10.0

6.7574396 7.5 7.5
7.146657 7.5 10.0
8.529603 7.5 5.0
7.1334844 7.5 7.5
5.8378696 5 2.5
7.2832446 7.5 7.5
7.612223 7.5 7.5
6.818888 7.5 7.5
7.420518 7.5 7.5
7.2980275 7.5 5.0
5.7383604 5 2.5
6.2615175 7.5 7.5
6.6522627 7.5 7.5
6.1226707 5 7.5
4.436254 5 7.5
7.765819 7.5 7.5
7.6464105 7.5 5.0
7.4550295 7.5 10.0
8.247253 7.5 7.5
7.783889 7.5 10.0
6.8527164 7.5 7.5
7.6722775 7.5 7.5
7.215664 7.5 2.5
6.639135 7.5 10.0
7.11952 7.5 7.5
8.538041 7.5 10.0
9.917143 10 10.0
7.593615 7.5 10.0
8.31717 7.5 7.5
7.4880567 7.5 2.5
6.509329 7.5 7.5
6.3432565 7.5 2.5
8.837619 10 10.0
7.251931 7.5 7.5
5.1459727 5 2.5
4.7939253 5 0.0
7.554099 7.5 10.0
5.7530193 5 2.5
7.1315784 7.5 10.0
6.6707196 7.5 7.5
9.495126 10 10.0
7.509618 7.5 10.0
7.072568 7.5 10.0
8.936038 10 10.0
7.7594514 7.5 7.5
7.373159 7.5 10.0
7.533154 7.5 10.0
9.063518 10 7.5
6.768366 7.5 2.5
6.259156 7.5 7.5
7.2441187 7.5 7.5
7.7757993 7.5 7.5
6.8099904 7.5 5.0
6.2015033 5 2.5
6.786014 7.5 5.0
8.877727 10 10.0
8.141465 7.5 5.0
8.738115 7.5 

5.795179 5 2.5
8.171051 7.5 5.0
5.4485793 5 2.5
6.559353 7.5 10.0
8.21508 7.5 5.0
8.887403 10 10.0
6.473309 7.5 5.0
9.495146 10 10.0
6.067431 5 7.5
6.878742 7.5 7.5
7.1737523 7.5 5.0
7.470629 7.5 7.5
7.115641 7.5 0.0
5.6777554 5 5.0
5.8339877 5 2.5
7.111055 7.5 7.5
6.7236257 7.5 10.0
7.230857 7.5 2.5
6.8952537 7.5 5.0
7.876106 7.5 10.0
9.333084 10 7.5
6.7052875 7.5 10.0
8.9069605 10 7.5
9.08294 10 10.0
8.777982 10 10.0
8.99087 10 10.0
7.303084 7.5 10.0
7.020656 7.5 7.5
5.3853273 5 5.0
8.794083 10 10.0
8.935527 10 10.0
7.2178936 7.5 7.5
8.520957 7.5 5.0
7.0320234 7.5 2.5
8.815841 10 7.5
9.146775 10 10.0
9.14814 10 10.0
6.1299076 5 0.0
8.279397 7.5 7.5
8.929781 10 10.0
7.00511 7.5 10.0
7.3344545 7.5 10.0
7.4628363 7.5 10.0
6.5345902 7.5 5.0
7.4571605 7.5 7.5
5.4562893 5 7.5
8.557975 7.5 7.5
7.588536 7.5 7.5
8.936083 10 10.0
7.102791 7.5 5.0
8.20962 7.5 10.0
7.629561 7.5 10.0
6.944566 7.5 5.0
7.9404235 7.5 7.5
6.6885858 7.5 7.5
7.023529 7.5 7.5
5.356553 5 2.5
9.079288 10 10.0
7.165941 7.5

7.760685 7.5 10.0
8.821927 10 7.5
7.4442616 7.5 7.5
7.7990084 7.5 7.5
6.3827243 7.5 2.5
7.2358866 7.5 10.0
8.875368 10 10.0
6.5619397 7.5 5.0
9.182947 10 10.0
4.367136 5 7.5
7.6697392 7.5 7.5
7.6056943 7.5 7.5
7.1915956 7.5 10.0
8.1389675 7.5 10.0
8.704581 7.5 7.5
7.4515715 7.5 7.5
7.3995833 7.5 2.5
5.835661 5 2.5
8.826144 10 10.0
9.235827 10 7.5
5.5191054 5 5.0
6.203065 5 7.5
6.5741863 7.5 7.5
6.2972455 7.5 10.0
7.064277 7.5 5.0
5.743789 5 5.0
8.2548685 7.5 10.0
8.869461 10 10.0
7.3615146 7.5 7.5
6.889896 7.5 7.5
6.8485804 7.5 7.5
6.5264893 7.5 7.5
6.994022 7.5 7.5
7.7897563 7.5 10.0
8.830721 10 7.5
6.8106136 7.5 10.0
7.553045 7.5 10.0
6.657207 7.5 5.0
7.0640388 7.5 7.5
6.8612704 7.5 5.0
8.883724 10 7.5
7.7433324 7.5 7.5
7.1755404 7.5 10.0
7.748843 7.5 7.5
6.334149 7.5 5.0
9.155457 10 10.0
8.379332 7.5 10.0
8.09853 7.5 10.0
8.055823 7.5 7.5
6.5188313 7.5 10.0
8.244404 7.5 7.5
7.3874316 7.5 5.0
7.6197114 7.5 7.5
5.0924687 5 2.5
7.3565054 7.5 7.5
6.8516836 7.5 5.0
7.22283 7.5 7.5
8.5440

In [51]:
print (mean_squared_error(y_test,clipS(xgb_preds))**.5)

2.35883188889119


In [74]:
import tensorflow as tf
import keras
import theano
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
tf.logging.set_verbosity(tf.logging.INFO)
np.random.seed(123)  # for reproducibility
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

np.random.seed(1337)  # for reproducibility

In [88]:
model = Sequential()
model.add(Dense(30, input_dim=346, activation='relu'))
#model.add(Dense(200, activation='relu'))
#model.add(Dense(100, activation='relu'))
#model.add(Dense(50, activation='relu'))
#model.add(Dense(25, activation='relu'))
#model.add(Dense(10, activation='relu'))
#model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='relu'))

model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(X_train, y_train, epochs=2, batch_size=10)

scores = model.evaluate(X_test, y_test)
print(scores**.5)

Epoch 1/2
11202/11202 [==============================] - 2s 220us/step - loss: 62.7304
Epoch 2/2
4802/4802 [==============================] - 0s 61us/step
7.92016863585144
